This Notebook is intended for a PhD subject of Econometric Methods in order to apply the Differences-in-Differences (DID) methodology.

The dataset is adapted from the dataset in [Card and Krueger (1994)](https://davidcard.berkeley.edu/papers/njmin-aer.pdf), which estimates the causal effect of an increase in the state minimum wage on the employment. The full dataset can be downloaded [here](https://www.kaggle.com/datasets/harrywang/propensity-score-matching).

The study at hand:
On April 1, 1992, New Jersey raised the state minimum wage from 4.25 USD to 5.05 USD while the minimum wage in Pennsylvania stays the same at 4.25 USD. Data about employment in fast-food restaurants in NJ (0) and PA (1) were collected in February 1992 and in November 1992 from 384 restaurants in total after removing null values.

In [3]:
import pandas as pd

In [5]:
df = pd.read_csv('propensity-score-matching/employment.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   state          384 non-null    int64  
 1   total_emp_feb  384 non-null    float64
 2   total_emp_nov  384 non-null    float64
dtypes: float64(2), int64(1)
memory usage: 9.1 KB


In [6]:
df.head()

,state,total_emp_feb,total_emp_nov
0,0,40.50,24.0
1,0,13.75,11.5
2,0,8.50,10.5
3,0,34.00,20.0
4,0,24.00,35.5


In [7]:
df.groupby('state').mean()

,total_emp_feb,total_emp_nov
state,,
0,23.380000,21.096667
1,20.430583,20.897249


In [8]:
# check by calculating the mean for each group directly
# 0 PA control group, 1 NJ treatment group

mean_emp_pa_before = df.groupby('state').mean().iloc[0, 0]
mean_emp_pa_after = df.groupby('state').mean().iloc[0, 1]
mean_emp_nj_before = df.groupby('state').mean().iloc[1, 0]
mean_emp_nj_after = df.groupby('state').mean().iloc[1, 1]

print(f'mean PA employment before: {mean_emp_pa_before:.2f}')
print(f'mean PA employment after: {mean_emp_pa_after:.2f}')
print(f'mean NJ employment before: {mean_emp_nj_before:.2f}')
print(f'mean NJ employment after: {mean_emp_nj_after:.2f}')

pa_diff = mean_emp_pa_after - mean_emp_pa_before
nj_diff = mean_emp_nj_after - mean_emp_nj_before
did = nj_diff - pa_diff

print(f'DID in mean employment is {did:.2f}')

mean PA employment before: 23.38
mean PA employment after: 21.10
mean NJ employment before: 20.43
mean NJ employment after: 20.90
DID in mean employment is 2.75


The calculation of DID is simple:

- mean PA (control group) employee per restaurant before/after the treatment is 23.38/21.1, so the after/before difference for the control group is -2.28 (21.1 - 23.38)
- mean NJ (treatment group) employee per restaurant before/after the treatment is 20.43/20.90, so the after/before difference for the treatment group is 0.47 (20.9 - 20.43)
- the difference-in-differences (DID) is 2.75 (0.47 + 2.28), which is (the after/before difference of the treatment group) - (the after/before difference of the control group)

The same DID result can be obtained via regression, which allows adding control variables if needed:

$y = \beta_0 + \beta_1 * g + \beta_2 * t + \beta_3 * (t * g) + \varepsilon$

- g is 0 for the control group and 1 for the treatment group
- t is 0 for before and 1 for after

we can insert the values of g and t using the table below and see that coefficient ($\beta_3$) of the interaction of g and t is the value for DID：

|              | Control Group (g=0) | Treatment Group (g=1)                   |                 |
|--------------|---------------------|-----------------------------------------|-----------------|
| Before (t=0) | $\beta_0$           | $\beta_0 + \beta_1$                     |                 |
| After (t=1)  | $\beta_0 + \beta_2$ | $\beta_0 + \beta_1 + \beta_2 + \beta_3$ |                 |
| Difference   | $\beta_2$           | $\beta_2 + \beta_3$                     | $\beta_3$ (DID) |


In [10]:
# group g: 0 control group (PA), 1 treatment group (NJ)
# t: 0 before treatment (min wage raise), 1 after treatment
# gt: interaction of g * t

# data before the treatment
df_before = df[['total_emp_feb', 'state']]
df_before['t'] = 0
df_before.columns = ['total_emp', 'g', 't']

# data after the treatment
df_after = df[['total_emp_nov', 'state']]
df_after['t'] = 1
df_after.columns = ['total_emp', 'g', 't']

# data for regression
df_reg = pd.concat([df_before, df_after])

# create the interaction 
df_reg['gt'] = df_reg.g * df_reg.t

df_reg

,total_emp,g,t,gt
0,40.50,0,0,0
1,13.75,0,0,0
2,8.50,0,0,0
3,34.00,0,0,0
4,24.00,0,0,0
...,...,...,...,...
379,23.75,1,1,1
380,17.50,1,1,1
381,20.50,1,1,1
382,20.50,1,1,1


In [11]:
# regression via sklearn
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

X = df_reg[['g', 't', 'gt']]
y = df_reg.total_emp

lr.fit(X, y)
lr.coef_  # the coefficient for gt is the DID, which is 2.75

array([-2.94941748, -2.28333333,  2.75      ])

In [12]:
# regression via statsmodels
# result is not significant 

from statsmodels.formula.api import ols
ols = ols('total_emp ~ g + t + gt', data=df_reg).fit()
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:              total_emp   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.947
Date:                Mon, 22 May 2023   Prob (F-statistic):              0.121
Time:                        18:34:40   Log-Likelihood:                -2817.6
No. Observations:                 768   AIC:                             5643.
Df Residuals:                     764   BIC:                             5662.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     23.3800      1.098     21.288      0.0


The p-value for $\beta_3$ in this example is not significant, which means that the average total employees per restaurant increased after the minimal salary raise by 2.75 FTE (full-time equivalent) but the result may be just due to random factors.